In [20]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pandas as pd
import torch
import gc
import Stemmer
import re
from tqdm import tqdm
import csv
import gc
import networkx as nx
from utils.preprocessing import preprocess_text

In [2]:
fbqa = pd.read_json("/datasets/FreebaseQA/FreebaseQA-eval.json")
fbqa=pd.json_normalize(fbqa.Questions)
fbqa.head()

,Dataset,Version,Questions
0,FreebaseQA-eval,1,"{'Question-ID': 'FreebaseQA-eval-0', 'RawQuest..."
1,FreebaseQA-eval,1,"{'Question-ID': 'FreebaseQA-eval-1', 'RawQuest..."
2,FreebaseQA-eval,1,"{'Question-ID': 'FreebaseQA-eval-2', 'RawQuest..."
3,FreebaseQA-eval,1,"{'Question-ID': 'FreebaseQA-eval-3', 'RawQuest..."
4,FreebaseQA-eval,1,"{'Question-ID': 'FreebaseQA-eval-4', 'RawQuest..."


In [3]:
fbqa.iloc[0].Questions

{'Question-ID': 'FreebaseQA-eval-0',
 'RawQuestion': "Who is the female presenter of the Channel 4 quiz show '1001 things you should know'?",
 'ProcessedQuestion': "who is the female presenter of the channel 4 quiz show '1001 things you should know'",
 'Parses': [{'Parse-Id': 'FreebaseQA-eval-0.P0',
   'PotentialTopicEntityMention': '1001 things you should know',
   'TopicEntityName': '1001 things you should know',
   'TopicEntityMid': 'm.0nd3t34',
   'InferentialChain': 'tv.tv_program.regular_personal_appearances..tv.tv_regular_personal_appearance.person',
   'Answers': [{'AnswersMid': 'm.0216y_', 'AnswersName': ['sandi toksvig']}]}]}

In [4]:
data = []
for i, r in fbqa.iterrows():
    question = r.Questions.get("RawQuestion")
    qid = r.Questions.get("Question-ID")
    parses = r.Questions.get("Parses", [])
    for parse in parses:
        if not parse:
            print(f"error in question: {question}")
            continue
        chain = parse.get("InferentialChain")
        entity = parse.get("TopicEntityName")
        entity_mid = parse.get("TopicEntityMid")
        answers = parse.get("Answers")
        for answer in answers:
            answer_mid = answer.get("AnswersMid")
            answer_name = answer.get("AnswersName")
            data.append([
                qid, question, entity, entity_mid, 
                answer_mid, answer_name, chain,
                ])
items = pd.DataFrame(columns=["qid", "question", "entity", "entity_mid", "answer_mid", "answer", "chain"], data=data)
# reformats the mid's to match with kb
items["entity_mid"] = items.apply(lambda m: f".{m['entity_mid']}".replace(".", "/"), axis=1)
items["answer_mid"] = items.apply(lambda m: f".{m['answer_mid']}".replace(".", "/"), axis=1)
items.head()

,qid,question,entity,entity_mid,answer_mid,answer,chain
0,FreebaseQA-eval-0,Who is the female presenter of the Channel 4 q...,1001 things you should know,/m/0nd3t34,/m/0216y_,[sandi toksvig],tv.tv_program.regular_personal_appearances..tv...
1,FreebaseQA-eval-1,"Who produced the film 12 Angry Men, which was ...",12 angry men,/m/0m_tj,/m/0cj8x,[henry fonda],film.film.produced_by
2,FreebaseQA-eval-2,Who directed the films; The Fisher King (1991)...,12 monkeys,/m/07j6w,/m/07h5d,[terry gilliam],film.film.directed_by
3,FreebaseQA-eval-2,Who directed the films; The Fisher King (1991)...,the brothers grimm,/m/04z257,/m/07h5d,[terry gilliam],film.film.directed_by
4,FreebaseQA-eval-2,Who directed the films; The Fisher King (1991)...,fisher king,/m/016z43,/m/07h5d,[terry gilliam],film.film.directed_by


In [175]:
####### create a graph
from utils.graph import KGraphPreproc
fbkb_graph = KGraphPreproc.get_fbkb_graph()

In [71]:
shortest_paths = dict(nx.all_pairs_shortest_path_length(fbkb_graph._graph))

In [8]:
# import json
# # serialize dict for caching
# with open("/datasets/FreebaseQA/cache/graph_paths.json", "w") as cache_file:
#     json.dump(shortest_paths, cache_file)


In [9]:
# with open("/datasets/FreebaseQA/cache/graph_paths.json", "r") as cache_file:
#     json.load(cache_file)

In [10]:
list(shortest_paths.keys())[:5]

['/m/027rn', '/m/06cx9', '/m/017dcd', '/m/06v8s0', '/m/07s9rl0']

In [72]:
def get_n_hops(q_row):
    start = q_row["entity_mid"]
    end = q_row["answer_mid"]
    return shortest_paths.get(start, {}).get(end, -1)

In [73]:
items["hops"] = items.apply(get_n_hops, axis=1)
print(items.groupby(by="hops").size())
items.head()

hops
-1    6027
 1    1160
 2      75
 3      38
 4       1
dtype: int64


,qid,question,entity,entity_mid,answer_mid,answer,chain,hops
0,FreebaseQA-eval-0,Who is the female presenter of the Channel 4 q...,1001 things you should know,/m/0nd3t34,/m/0216y_,[sandi toksvig],tv.tv_program.regular_personal_appearances..tv...,-1
1,FreebaseQA-eval-1,"Who produced the film 12 Angry Men, which was ...",12 angry men,/m/0m_tj,/m/0cj8x,[henry fonda],film.film.produced_by,-1
2,FreebaseQA-eval-2,Who directed the films; The Fisher King (1991)...,12 monkeys,/m/07j6w,/m/07h5d,[terry gilliam],film.film.directed_by,-1
3,FreebaseQA-eval-2,Who directed the films; The Fisher King (1991)...,the brothers grimm,/m/04z257,/m/07h5d,[terry gilliam],film.film.directed_by,1
4,FreebaseQA-eval-2,Who directed the films; The Fisher King (1991)...,fisher king,/m/016z43,/m/07h5d,[terry gilliam],film.film.directed_by,1


### FBQA hop numbers
after comparing the number of hops with the KB, the number of hops is actually different. Makes sense, as the number of edges in the inferential chain is not the same as neighbor distance in the graph.


### Next steps:
1. Recompute the baseline and KB1 results, filter by question and recompute statistics.
2. Similarity-based graph traversal (top 5 branches at a time)

In [176]:
from utils.evaluation import FBQA_Dataset
fbqa_eval = FBQA_Dataset()
results = fbqa_eval.results
results["qid"] = fbqa_eval.test_set.Questions.apply(lambda t: t.get("Question-ID"))
results[:2]


,Actual,bline,bline2,kb1,quid,qid
0,sandi toksvig,Anita Rani,Anna Richardson,Anna Richardson,FreebaseQA-eval-0,FreebaseQA-eval-0
1,henry fonda,Mervyn LeRoy produced the film 12 Angry Men.,Mervyn LeRoy produced the film 12 Angry Men.,Sidney Lumet,FreebaseQA-eval-1,FreebaseQA-eval-1


In [177]:
results_df = items[["qid", "hops"]].merge(results, on="qid", how="left")
results_df[:2]

,qid,hops,Actual,bline,bline2,kb1,quid
0,FreebaseQA-eval-0,-1,sandi toksvig,Anita Rani,Anna Richardson,Anna Richardson,FreebaseQA-eval-0
1,FreebaseQA-eval-1,-1,henry fonda,Mervyn LeRoy produced the film 12 Angry Men.,Mervyn LeRoy produced the film 12 Angry Men.,Sidney Lumet,FreebaseQA-eval-1


In [178]:
bmarks = ["bline", "bline2", "kb1"]
for bmark in bmarks:
    results_df[f"{bmark}_correct"] = results_df.apply(
        lambda t: fbqa_eval.check_correct(t[bmark], t["Actual"]), 
        axis=1
    )
results_df[:2]

,qid,hops,Actual,bline,bline2,kb1,quid,bline_correct,bline2_correct,kb1_correct
0,FreebaseQA-eval-0,-1,sandi toksvig,Anita Rani,Anna Richardson,Anna Richardson,FreebaseQA-eval-0,False,False,False
1,FreebaseQA-eval-1,-1,henry fonda,Mervyn LeRoy produced the film 12 Angry Men.,Mervyn LeRoy produced the film 12 Angry Men.,Sidney Lumet,FreebaseQA-eval-1,False,False,False


In [78]:
results_df.groupby(by="hops")[[f"{bmark}_correct" for bmark in bmarks]].agg(["size", "sum"])

bline_correct       bline2_correct       kb1_correct      
              size   sum           size   sum        size   sum
hops                                                           
-1            6027  4874           6027  4916        6027  4370
 1            1160  1036           1160  1032        1160   971
 2              75    67             75    67          75    57
 3              38    30             38    29          38    26
 4               1     1              1     1           1     1

In [257]:
results_df[results_df.hops == 3].iloc[0]

qid                                             FreebaseQA-eval-178
hops                                                              3
Actual                                              john f. kennedy
bline                                               John F. Kennedy
bline2                                              John F. Kennedy
kb1               John F. Kennedy was the first Roman Catholic t...
quid                                            FreebaseQA-eval-178
bline_correct                                                  True
bline2_correct                                                 True
kb1_correct                                                    True
Name: 340, dtype: object

In [179]:
results_df[(results_df.hops == 1) & (~results_df["kb1_correct"])][:2]

,qid,hops,Actual,bline,bline2,kb1,quid,bline_correct,bline2_correct,kb1_correct
30,FreebaseQA-eval-14,1,poland,Mexico (Men's Football),Mexico (men's team),The context does not provide information about...,FreebaseQA-eval-14,False,False,False
31,FreebaseQA-eval-14,1,poland,Mexico (Men's Football),Mexico (men's team),The context does not provide information about...,FreebaseQA-eval-14,False,False,False


In [258]:
# 
qid = "FreebaseQA-eval-178"
q = fbqa[fbqa["Question-ID"] == qid].iloc[0].RawQuestion
print(q)
fbqa[fbqa["Question-ID"] == qid].Parses.iloc[0]

Who was the first Roman Catholic to become American President?


[{'Parse-Id': 'FreebaseQA-eval-178.P0',
  'PotentialTopicEntityMention': 'american president',
  'TopicEntityName': 'president of the united states',
  'TopicEntityMid': 'm.060d2',
  'InferentialChain': 'base.firsts.achievement.firsts..base.firsts.first_achievement.first',
  'Answers': [{'AnswersMid': 'm.0d3k14', 'AnswersName': ['john f. kennedy']}]}]

In [149]:
triplets = fbkb_graph._get_entity_knowledge("/m/0c9k8", depth=1)
triplets

['Reds /film/film/genre Drama',
 'Reds /award/award_winning_work/awards_won./award/award_honor/award BAFTA_Award_for_Best_Actor_in_a_Supporting_Role',
 'Reds /award/award_winning_work/awards_won./award/award_honor/award_winner Jack_Nicholson',
 'Reds /award/award_category/nominees./award/award_nomination/nominated_for Academy_Award_for_Best_Production_Design',
 'Reds /award/award_category/nominees./award/award_nomination/nominated_for Academy_Award_for_Best_Sound_Mixing',
 'Reds /award/award_category/nominees./award/award_nomination/nominated_for Academy_Award_for_Best_Actor',
 'Reds /award/award_winning_work/awards_won./award/award_honor/award_winner Diane_Keaton',
 'Reds /award/award_category/nominees./award/award_nomination/nominated_for Golden_Globe_Award_for_Best_Screenplay_-_Motion_Picture',
 'Reds /award/award_category/nominees./award/award_nomination/nominated_for BAFTA_Award_for_Best_Actress_in_a_Leading_Role',
 'Reds /film/film/production_companies Paramount_Pictures',
 'Reds

### Reranking based on semantic similarity (embeddings)


In [147]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

In [150]:
triplet_embeddings = model.encode(triplets)
q_embedding = model.encode(q)
qt_sim = model.similarity(q_embedding, triplet_embeddings)
# list(zip(qt_sim[0], triplets))
qt_sim

tensor([[0.3808, 0.1630, 0.1757, 0.1302, 0.1316, 0.1847, 0.1878, 0.1998, 0.1991,
         0.3598, 0.2083, 0.1740, 0.3529, 0.1899, 0.1839, 0.1854, 0.1502, 0.4192,
         0.3502, 0.2250, 0.2300, 0.1209, 0.2494, 0.1478, 0.1141, 0.1955, 0.4301,
         0.1669, 0.1638, 0.2914, 0.2163, 0.6508, 0.1053, 0.3684, 0.1896, 0.4720,
         0.3249, 0.2582, 0.1575, 0.3615, 0.3228, 0.4357, 0.2880, 0.3194, 0.1381,
         0.1993, 0.4239, 0.2730, 0.3933, 0.3424, 0.2021, 0.1678, 0.1233, 0.1558,
         0.3547]])